<a href="https://colab.research.google.com/github/cagBRT/computer-vision/blob/master/CV1_ObjectLocalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -l -s https://github.com/cagBRT/computer-vision.git cloned-repo
%cd cloned-repo

In [ ]:
from IPython.display import Image
def page(num):
    return Image("/content/cloned-repo/pages/objectLocalization"+str(num)+ ".png" , width=640)

In [ ]:
page("")

In [ ]:
page(1)

**Import the libraries**

In [ ]:
 %tensorflow_version 2.x
 import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.utils import  plot_model
import matplotlib.pyplot as plt

BATCH_SIZE = 64
EPOCH_SIZE = 64

**Load the pretrained VGG16 model**<br>
You may recall this from the introduction notebook "Introduction to Deep Neural Networks with Keras". This same architecture was used for image classification of hand drawn digits. 

In [ ]:
page(2)

**Feature extraction**<br>
Feature extraction is the process of showing an ML model small pieces of an image and training it to detect the whole. <br>
For example: <BR>
Extract nine features from an image of a motorcycle and ask the model to determine what object is in the image. 

In [ ]:
Image("/content/cloned-repo/images/motorcycle.png" , width=640)

In [ ]:
# transfer learning - load pre-trained vgg and replace its head
vgg = tf.keras.applications.VGG16(input_shape=[128, 128, 3], include_top=False, weights='imagenet')
x = Flatten()(vgg.output)
x = Dense(3, activation='sigmoid')(x)
model1 = Model(vgg.input, x)
model1.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001))
# plot the model
plot_model(model1, "first_model.png",show_shapes=True,expand_nested=False)

In [ ]:
model1.summary()

**Create a synthetic dataset**

In [ ]:
from matplotlib.patches import Circle
#Our dataset is called synthetic_gen

def synthetic_gen(batch_size=64):
  # enable generating infinite amount of batches
  while True:
      # generate black images in the wanted size
      X = np.zeros((batch_size, 128, 128, 3))
      Y = np.zeros((batch_size, 3))
      # fill each image
      for i in range(batch_size):
        x = np.random.randint(8,120)
        y = np.random.randint(8,120)
        a = min(128 - max(x,y), min(x,y))
        r = np.random.randint(4,a)
        for x_i in range(128):
          for y_i in range(128):
            if ((x_i - x)**2) + ((y_i - y)**2) < r**2:
              X[i, x_i, y_i,:] = 1
        Y[i,0] = (x-r)/128.
        Y[i,1] = (y-r)/128.
        Y[i,2] = 2*r / 128.
      yield X, Y

# sanity check - plot the images
x,y = next(synthetic_gen())
plt.imshow(x[0])

In [ ]:
plt.imshow(x[63])

**Train the model to predict the bounding box** for each white circle on the black background

In [ ]:
# needs steps per epoch since the generator is infinite
model1.fit_generator(synthetic_gen(),steps_per_epoch=EPOCH_SIZE,epochs=5)

**Add bounding boxes**<br>
When an object is detected, <br>
>get the coordinates for the upper left corner of the bounding box<br>
get the width and height of the bounding box<br>

>The draw the bounding box on the image

In [ ]:
from matplotlib.patches import Rectangle

# given image and a label, plots the image + rectangle
def plot_pred(img,p):
  fig, ax = plt.subplots(1)
  ax.imshow(img)
  rect = Rectangle(xy=(p[1]*128,p[0]*128),width=p[2]*128, height=p[2]*128, 
                   linewidth=3,edgecolor='g',facecolor='none')
  ax.add_patch(rect)
  plt.show()


# generate new image
x, _ = next(synthetic_gen())
# predict
pred = model1.predict(x)
# examine 1 image
im = x[0]
p = pred[0]
plot_pred(im,p)

In [ ]:
im = x[20]
p = pred[20]
plot_pred(im,p)

Circles are easy, let's try a different type of image. <br>


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
image = cv2.imread("/content/cloned-repo/cat.png")
cv2_imshow(image)

**Cat Images**<br>
Place the cat image on a black background, can the model find the image and can an accurate box be drawn around the cat?

In [ ]:
from PIL import Image
from matplotlib.patches import Circle

cat_pil = Image.open("cat.png")
cat_pil = cat_pil.resize((64,64))
cat = np.asarray(cat_pil)

#define a function to generate different versions of the cat image
def cat_gen(batch_size=64):
  # enable generating infinite amount of batches
  while True:
      # generate black images in the wanted size
      X = np.zeros((batch_size, 128, 128, 3))
      Y = np.zeros((batch_size, 3))
      # fill each image
      for i in range(batch_size):
        # resize the cat
        size = np.random.randint(32,64)
        #To rotate the image, change the following line
        temp_cat = cat_pil.resize((size,size))
        cat = np.asarray(temp_cat) / 255.
        cat_x, cat_y, _ = cat.shape
        # create a blank background image
        bg = Image.new('RGB', (128, 128))
        # generate 
        x1 = np.random.randint(1,128 - cat_x)
        y1 = np.random.randint(1,128 - cat_y)
        # paste the cat over the image
        bg.paste(temp_cat, (x1, y1))
        cat = np.asarray(bg) / 255. # transform into a np array
        X[i] = cat

        Y[i,0] = x1/128.
        Y[i,1] = y1/128.
        Y[i,2] = cat_x / 128.
      yield X, Y

# plot the images
x,y = next(cat_gen())
plt.imshow(x[32])

**Use the pretrained VGG16 model**<br>
The pretrained model is now used to find the cat on a black background. 


In [ ]:
# transfer learning - load pre-trained vgg and replace its head
vgg = tf.keras.applications.VGG16(input_shape=[128, 128, 3], include_top=False, weights='imagenet')
x = Flatten()(vgg.output)
x = Dense(3, activation='sigmoid')(x)
model2 = Model(vgg.input, x)
model2.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001))
# plot the model
plot_model(model2, "second_model.png",show_shapes=True)

# needs steps per epoch since the generator is infinite
model2.fit_generator(cat_gen(),steps_per_epoch=EPOCH_SIZE,epochs=5)

from matplotlib.patches import Rectangle

# given image and a label, plots the image + rectangle
def plot_pred(img,p):
  fig, ax = plt.subplots(1)
  ax.imshow(img)
  rect = Rectangle(xy=(p[0]*128,p[1]*128),width=p[2]*128, height=p[2]*128, 
                   linewidth=4,edgecolor='g',facecolor='none')
  ax.add_patch(rect)
  plt.show()

**Generate a new image with the bounding box**<br>
After finding the cat, draw a bounding box around it. 

In [ ]:
# generate new image
x, _ = next(cat_gen())
# predict
pred = model2.predict(x)
# examine 1 image
im = x[0]
p = pred[0]
plot_pred(im,p)

In [ ]:
num=10
im = x[num]
p = pred[num]
plot_pred(im,p)

**Can the model find the object if we rotate it?**<br>
The model found the cat when the cat was in the same orientation as the original, does it matter if the orientation is changed?

# **Assignment 1** 
Rotate the image at least 30 degrees, can the model still find the image?<br>
1. Generate in cat images with rotation instead of resize
2. Make predictions
3. Display the image with the bounding box

*Hint: You don't need to retrain the model*

In [ ]:
#Assignment 1 - Rotate the object
cat_pil = Image.open("cat.png")
cat_pil = cat_pil.resize((64,64))
cat = np.asarray(cat_pil)

#define a function to generate different versions of the cat image
def cat_gen(batch_size=64):
  # enable generating infinite amount of batches
  while True:
      # generate black images in the wanted size
      X = np.zeros((batch_size, 128, 128, 3))
      Y = np.zeros((batch_size, 3))
      # fill each image
      for i in range(batch_size):
        # resize the cat
        size = np.random.randint(32,64)
        #To rotate the image, change the following line
        temp_cat = cat_pil.resize((size,size))
        #===Rotate code here===#
        
        cat = np.asarray(temp_cat) / 255.
        cat_x, cat_y, _ = cat.shape
        # create a blank background image
        bg = Image.new('RGB', (128, 128))
        # generate 
        x1 = np.random.randint(1,128 - cat_x)
        y1 = np.random.randint(1,128 - cat_y)
        # paste the cat over the image
        bg.paste(temp_cat, (x1, y1))
        cat = np.asarray(bg) / 255. # transform into a np array
        X[i] = cat

        Y[i,0] = x1/128.
        Y[i,1] = y1/128.
        Y[i,2] = cat_x / 128.
      yield X, Y

# plot the images
x,y = next(cat_gen())
plt.imshow(x[32])

x, _ = next(cat_gen())
# predict
pred = model2.predict(x)
# examine 1 image
im = x[0]
p = pred[0]
plot_pred(im,p)

In [ ]:
#@title
def cat_gen(batch_size=64):
  # enable generating infinite amount of batches
  while True:
      # generate black images in the wanted size
      X = np.zeros((batch_size, 128, 128, 3))
      Y = np.zeros((batch_size, 3))
      # fill each image
      for i in range(batch_size):
        # resize the cat
        size = np.random.randint(32,64)
        temp_cat = cat_pil.rotate(30)
        #temp_cat = cat_pil.resize((size,size))
        cat = np.asarray(temp_cat) / 255.
        cat_x, cat_y, _ = cat.shape
        # create a blank background image
        bg = Image.new('RGB', (128, 128))
        # generate 
        x1 = np.random.randint(1,128 - cat_x)
        y1 = np.random.randint(1,128 - cat_y)
        # paste the cat over the image
        bg.paste(temp_cat, (x1, y1))
        cat = np.asarray(bg) / 255. # transform into a np array
        X[i] = cat

        Y[i,0] = x1/128.
        Y[i,1] = y1/128.
        Y[i,2] = cat_x / 128.
      yield X, Y

# plot the images
x,y = next(cat_gen())
plt.imshow(x[32])

In [ ]:
#@title
# generate new image
x, _ = next(cat_gen())
# predict
pred = model2.predict(x)
# examine 1 image
im = x[0]
p = pred[0]
plot_pred(im,p)

**Can the model find the cat on a background that is not black?**

Create new images with the cat and landscape backgrounds

In [ ]:
from PIL import Image

cat_pil = Image.open("cat.png")
cat_pil = cat_pil.resize((64,64))
cat = np.asarray(cat_pil)

def natural_cat_gen(batch_size=64):
  # enable generating infinite amount of batches
  while True:
      # generate black images in the wanted size
      X = np.zeros((batch_size, 128, 128, 3))
      Y = np.zeros((batch_size, 3))
      # fill each image
      for i in range(batch_size):
        # resize the cat
        size = np.random.randint(32,64)
        temp_cat = cat_pil.resize((size,size))
        cat = np.asarray(temp_cat) / 255.
        cat_x, cat_y, _ = cat.shape
        # background image
        bg_name = f'landscape1.jpg'
        bg = Image.open(bg_name)

        x1 = np.random.randint(1,128 - cat_x)
        y1 = np.random.randint(1,128 - cat_y)
        h = cat_x
        w = cat_y
        # draw the cat over the selected background image
        bg.paste(temp_cat, (x1, y1),mask=temp_cat)
        cat = np.asarray(bg) / 255.
        X[i] = cat

        Y[i,0] = x1/128.
        Y[i,1] = y1/128.
        Y[i,2] = cat_x / 128.
      yield X, Y

# sanity check - plot the images
x,y = next(natural_cat_gen())
plt.imshow(x[0])

Use the pretrained VGG16 on the new background images

In [ ]:
# define a mode
# transfer learning - load pre-trained vgg and replace its head
vgg = tf.keras.applications.VGG16(input_shape=[128, 128, 3], include_top=False, weights='imagenet')
x = Flatten()(vgg.output)
x = Dense(3, activation='sigmoid')(x)
model3 = Model(vgg.input, x)
model3.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001))
# plot the model
plot_model(model3, "third_model.png",show_shapes=True)

# train it
# needs steps per epoch since the generator is infinite
model3.fit_generator(natural_cat_gen(),steps_per_epoch=EPOCH_SIZE,epochs=5)

from matplotlib.patches import Rectangle

# given image and a label, plots the image + rectangle
def plot_pred(img,p):
  fig, ax = plt.subplots(1)
  ax.imshow(img)
  rect = Rectangle(xy=(p[0]*128,p[1]*128),width=p[2]*128, height=p[2]*128, linewidth=1,edgecolor='r',facecolor='none')
  ax.add_patch(rect)
  plt.show()

Look at the images with the added bounding box

In [ ]:
# generate new image
x, _ = next(natural_cat_gen())
# predict
pred = model3.predict(x)
# examine 1 image
im = x[0]
p = pred[0]
plot_pred(im,p)

# **Assignment 2:** 
Change the background images and try the model on different backgrounds. 

In [ ]:
#Assignment 2
